# Pong

## What is Pong?

Pong is a table tennis–themed arcade sports video game, featuring simple two-dimensional graphics. The game was originally manufactured by Atari, which released it in 1972. Allan Alcorn created Pong as a training exercise assigned to him by Atari co-founder Nolan Bushnell. Atari made it famous, but it was actually copied from a game developed by Magnavox. It resulted in a lawsuit that was settled in 1976, with Atari paying Magnavox some monies.

## Introduction

In this tutorial, we will build a simple Pong game step-by-step using Python's Tkinter library.
Each step will add new functionality, from creating the canvas and ball to adding paddle controls, 
collision detection, and game management. Let's get started!

## Step 1: Create the Canvas {.hide .smaller-85}

**Step 1**: First, we’ll create a window and add a canvas where the game will take place. This will be the main area where the ball and paddle will interact.


```python
from tkinter import *

# Create the main window and canvas
tk = Tk()
tk.resizable(False, False)  # Prevent window resizing
tk.title("Pong Game")
canvas = Canvas(tk, width=600, height=500, bd=0, bg='ivory')
canvas.pack() # Pack is used to display objects in the window
canvas.update() # Needed to get the rendered canvas size 

# Run the main loop
tk.mainloop()
```

## Step 2: Create a Stationary Ball {.hide .smaller-85}

**Step 2**:  We define a `Ball` class to render the ball.  For now, the ball will be stationary in the middle of the canvas, 10% from the top.

```python
class Ball:
    def __init__(self, canvas, color):
        self.canvas = canvas
        self.canvas_width = self.canvas.winfo_width()
        self.canvas_height = self.canvas.winfo_height()
        # Create an oval (ball) with size 15x15 pixels
        self.id = canvas.create_oval(0, 0, 15, 15, fill=color)
        # Move it to the middle of the canvas, 10% from the top
        self.canvas.move(self.id, self.canvas_width/2, self.canvas_height*0.1)
```

Now we add a `Ball` instance to the canvas, do this below the canvas creation code:

```{.python code-line-numbers="4-5"}
canvas.pack()
canvas.update() # Needed to get the rendered canvas size  

# Create the ball
ball = Ball(canvas, 'red')

# Run the main loop
tk.mainloop()
```

## Step 3a: Ball Motion with Wall Detection {.hide .smaller-85}

**Step 3a**: For ball motion, we start by adding horizontal and vertical speed attributes to the `init` method of the `Ball` class.

```python
# Set the initial speed of the ball
self.xspeed = 2
self.yspeed = -2
```

Next, we add a `draw` method to the `Ball` class to update the ball's position and detect collisions with the canvas walls.

```python
def draw(self):
    # Move the ball by its speed values
    self.canvas.move(self.id, self.xspeed, self.yspeed)
    pos = self.canvas.coords(self.id)
    
    # Wall collision detection
    if pos[1] <= 0:    # Top wall
        self.yspeed = abs(self.yspeed)
    if pos[3] >= self.canvas_height:  # Bottom wall
        self.yspeed = -1*abs(self.yspeed)
    if pos[0] <= 0:    # Left wall
        self.xspeed = abs(self.xspeed)
    if pos[2] >= self.canvas_width:  # Right wall
        self.xspeed = -1*abs(self.xspeed)
```

## Step 3b: Animating the Ball {.hide .smaller-85}

**Step 3b**: Finally, create an `animate` function that updates and redraws the ball every 10ms and call it in the main loop.  Place this below the instantiation of the `Ball` object.

```{.python code-line-numbers="4-9"}
# Create the ball
ball = Ball(canvas, 'red')

# Animation loop
def animate():
    ball.draw()
    tk.after(10, animate)  # Schedule next update in 10ms

animate()  # Start animation
tk.mainloop()
```

## Step 4: Create the Paddle {.hide .smaller-85}

**Step 4**: Now, we’ll create the paddle, which the player will control to keep the ball in play.  The paddle will be a rectangle and only move horizontally, it will be at rest initially.

```python
class Paddle:
    def __init__(self, canvas, color):
        self.canvas = canvas
        self.canvas_width = self.canvas.winfo_width()
        self.canvas_height = self.canvas.winfo_height()
        self.id = canvas.create_rectangle(0, 0, 100, 10, fill=color)
        self.canvas.move(self.id, self.canvas_width/2, self.canvas_height*0.7)
        self.xspeed = 0
```

Add a paddle instance to the canvas, do this **above** the ball creation code:

```{.python code-line-numbers="1-2"}
# Create the paddle
paddle = Paddle(canvas, 'blue')

# Create the ball
ball = Ball(canvas, 'red')
```

## Step 5a: Paddle Motion {.hide .smaller-85}


**Step 5a**: We’ll now bind left and right movement to the paddle using the arrow keys on the keyboard. We need to add event handlers for key presses and releases, and methods to change the paddle speed.  Modify the `Paddle` class as follows:

```{.python code-line-numbers="7-15"}
class Paddle:
    def __init__(self, canvas, color):
        self.canvas = canvas
        self.id = canvas.create_rectangle(0, 0, 100, 10, fill=color)
        self.canvas.move(self.id, 200, 300)
        self.xspeed = 0
        # Bind keyboard controls
        self.canvas.bind_all('<KeyPress-Left>', self.move_left)
        self.canvas.bind_all('<KeyPress-Right>', self.move_right)
    
    def move_left(self, evt):
        self.xspeed = -2
    
    def move_right(self, evt):
        self.xspeed = 2
```

## Step 5b: Animate the Paddle {.hide .smaller-85}

**Step 5b**: Add a `draw` method to the `Paddle` class to animate the paddle.

```python
def draw(self):
    self.canvas.move(self.id, self.xspeed, 0)
    pos = self.canvas.coords(self.id)
    if pos[0] <= 0:
        self.xspeed = 0
    if pos[2] >= self.canvas_width:
        self.xspeed = 0
```

Finally, update the `animate` function to include the paddle animation.

```{.python code-line-numbers="3"}
def animate():
    ball.draw()
    paddle.draw()
    tk.after(10, animate)  # Schedule next update in 10ms
```

## Step 6a: Paddle Collision Detection {.hide .smaller-85}

**Step 6a**: Now, we’ll modify the `Ball` class to detect collisions with the paddle. First, add the `hit_paddle` method to the `Ball` class. 

```python
def hit_paddle(self, pos):
    paddle_pos = self.canvas.coords(self.paddle.id)
    if pos[2] >= paddle_pos[0] and pos[0] <= paddle_pos[2]:
        if pos[3] >= paddle_pos[1] and pos[3] <= paddle_pos[3]:
            return True
    return False
```

## Step 6b: Ball Collision Movement {.hide .smaller-85}

**Step 6b**: Next, modify the `draw` method of the `Ball` class to include paddle collision detection, and change the ball's vertical speed when it hits the paddle.


```{.python code-line-numbers="16-17"}
def draw(self):
    # Move the ball by its speed values
    self.canvas.move(self.id, self.xspeed, self.yspeed)
    pos = self.canvas.coords(self.id)
    
    # Wall collision detection
    if pos[1] <= 0:    # Top wall
        self.yspeed = abs(self.yspeed)
    if pos[3] >= self.canvas_height:  # Bottom wall
        self.yspeed = -1*abs(self.yspeed)
    if pos[0] <= 0:    # Left wall
        self.xspeed = abs(self.xspeed)
    if pos[2] >= self.canvas_width:  # Right wall
        self.xspeed = -1*abs(self.xspeed)

    if self.hit_paddle(pos):
        self.yspeed = -1*abs(self.yspeed)
```

## Step 7: Hit Bottom Wall Detection {.hide .smaller-85}


**Step 7**: We need to update the collision detection to detect if the ball hits the bottom wall, indicating the game is over. Add `hit_bottom` as an attribute to the `Ball` class, initilize it to `False`.

```python
self.hit_bottom = False
```

Modify the `draw` method to set `hit_bottom` to `True` when the ball hits the bottom wall.

```{.python code-line-numbers="9-10"}
def draw(self):
    # Move the ball by its speed values
    self.canvas.move(self.id, self.xspeed, self.yspeed)
    pos = self.canvas.coords(self.id)
    
    # Wall collision detection
    if pos[1] <= 0:    # Top wall
        self.yspeed = abs(self.yspeed)
    if pos[3] >= self.canvas_height:  # Bottom wall
        self.hit_bottom = True
    if pos[0] <= 0:    # Left wall
        self.xspeed = abs(self.xspeed)
    if pos[2] >= self.canvas_width:  # Right wall
        self.xspeed = -1*abs(self.xspeed)
```

## Step 8a: Game Over Functionality {.hide .smaller-85}

**Step 8**: Let's start by adding the functionality needed to let the user know when the game is over. We'll display a message on the canvas and allow the user to close the game by pressing the space key.

```python
def game_over():
    canvas.create_text(
        canvas.winfo_width()/2, canvas.winfo_height()/2,
        text="GAME OVER",
        font=("Helvetica", 30),
        fill='red'
    )
    canvas.create_text(
        canvas.winfo_width()/2, canvas.winfo_height()/2+50,
        text="Press Space to quit",
        font=("Helvetica", 20)
    )
    # Rebind space key to close the game
    tk.bind('<space>', close_game)

def close_game(event=None):
    tk.destroy()
```

## Step 8b: Game Over {.hide .smaller-85}

**Step 8b**: Finally, update the `animate` function to check if the `ball` hit the bottom wall and call the `game_over` function.



```{.python code-line-numbers="4-7"}
def animate():
    ball.draw()
    paddle.draw()
    if not ball.hit_bottom:
        tk.after(10, animate)  # Schedule next update in 10ms
    else:
        game_over()
```